# Calculating the Relative Strength Index and Testing a Long/Short Strategy

* The `Relative Strength Index (RSI)`, developed by J. Welles Wilder, is a momentum oscillator that measures the speed and change of price movements. \
The RSI oscillates between zero and 100. Traditionally the RSI is considered `overbought when above 70 and oversold when below 30`. \
Signals can be generated by looking for divergences and failure swings. RSI can also be used to identify the general trend. The basic formula is:

                    RSI = 100 – [100 / ( 1 + (Average of Upward Price Change / Average of Downward Price Change ) ) ]                                 (Fidelity Investments)

In [1]:
import backtrader as bt
import datetime
import matplotlib.pyplot as plt
import talib
import pandas as pd
import warnings
import yfinance as yf

# plt.style.use('seaborn')
# plt.style.use('seaborn-colorblind') #alternative
plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 300
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Download data
data = bt.feeds.PandasData(dataname=yf.download("KO", start="2020-01-01", end="2020-12-31"))

[*********************100%%**********************]  1 of 1 completed


1- Define the signal strategy based on bt.SignalStrategy

In [3]:
# create a Strategy
class RsiSignalStrategy(bt.SignalStrategy):
    params = dict(rsi_periods=14, rsi_upper=70, rsi_lower=30, rsi_mid=50)

    def __init__(self):
        # add RSI Indicator
        rsi = bt.indicators.RSI(period=self.p.rsi_periods, upperband=self.p.rsi_upper, lowerband=self.p.rsi_lower)

        # add RSI from TA-Lib just for reference
        bt.talib.RSI(self.data, plotname='TA_RSI')

        # Long condition (with exit)
        rsi_signal_long = bt.ind.CrossUp(rsi, self.p.rsi_lower, plot=False)
        self.signal_add(bt.SIGNAL_LONG, rsi_signal_long)
        self.signal_add(bt.SIGNAL_LONGEXIT, -(rsi > self.p.rsi_mid))

        # short condition (with exit)
        rsi_signal_short=bt.ind.CrossDown(rsi, self.p.rsi_upper, plot=False)
        self.signal_add(bt.SIGNAL_SHORT, rsi_signal_short)
        self.signal_add(bt.SIGNAL_SHORTEXIT, rsi < self.p.rsi_mid)

2- Set up and run the backtest

In [4]:
cerebro = bt.Cerebro(stdstats=False)

cerebro.adddata(data)
cerebro.addstrategy(RsiSignalStrategy)
cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.Value)

cerebro.run()

3- Plot Result

In [5]:
cerebro.plot(iplot=False, volume=False)

[[<Figure size 2100x1019 with 4 Axes>]]